In [1]:
import joblib
import pandas as pd
import numpy as np

In [6]:
data = joblib.load('16k_apperal_data_preprocessed')

In [7]:
data

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,$26.26
6,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,$9.99
15,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,$20.54
27,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,$7.39
46,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,$6.95
...,...,...,...,...,...,...,...
183081,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,$19.25
183092,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,$23.24
183096,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,$29.99
183101,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,$22.83


In [39]:
df = pd.read_csv('16k_apperal_data_preprocessed.csv')
df.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,$26.26
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,$9.99
2,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,$20.54
3,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,$7.39
4,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,$6.95


In [40]:
df.shape

(16042, 7)

In [42]:
df['formatted_price'] = df['formatted_price'].str.replace('$','')

In [43]:
df['formatted_price'] = df['formatted_price'].str.strip()

In [67]:
df['formatted_price'] = df['formatted_price'].apply(pd.to_numeric,errors='coerce')

In [68]:
df.dtypes

asin                  object
brand                 object
color                 object
medium_image_url      object
product_type_name     object
title                 object
formatted_price      float64
dtype: object

In [69]:
df.to_csv('aaic_task_data.csv',index=False)

In [13]:
dff = pd.read_csv('aaic_task_data.csv')
dff

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
2,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54
3,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39
4,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95
...,...,...,...,...,...,...,...
16037,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,19.25
16038,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,23.24
16039,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,29.99
16040,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,22.83


In [15]:
dff['id'] = range(1,dff.shape[0]+1)

In [17]:
dff.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price,id
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26,1
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99,2
2,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54,3
3,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39,4
4,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95,5


In [52]:
dff['product_type_name'].value_counts()

SHIRT                               12827
BOOKS_1973_AND_LATER                 1334
APPAREL                               620
ACCESSORY                             302
SPORTING_GOODS                        194
DRESS                                 141
BLAZER                                 93
SWEATER                                79
OUTERWEAR                              69
OUTDOOR_RECREATION_PRODUCT             53
UNDERWEAR                              35
SLEEPWEAR                              34
MISC_OTHER                             33
HEALTH_PERSONAL_CARE                   33
PANTS                                  22
SKIRT                                  20
AUTO_ACCESSORY                         20
TOYS_AND_GAMES                         20
ETHNIC_WEAR                            16
HANDBAG                                 9
POWERSPORTS_PROTECTIVE_GEAR             9
BRA                                     8
BEAUTY                                  6
SHOES                             

In [3]:
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [77]:
c.execute('''DROP TABLE IF EXISTS aaic;''')

c.execute('''CREATE TABLE aaic(
    id INTEGER PRIMARY KEY,
    asin VARCHAR(15),
    brand VARCHAR(100),
    color VARCHAR(20),
    medium_image_url TEXT,
    product_type_name VARCHAR(20),
    title TEXT,
    formatted_price VARCHAR(5));''')

In [84]:
dff.to_sql('aaic', conn, if_exists='append', index = False)

In [4]:
pd.read_sql('''SELECT * FROM aaic ''', conn)

,id,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,1,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26
1,2,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
2,3,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54
3,4,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39
4,5,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95
...,...,...,...,...,...,...,...,...
16037,16038,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,19.25
16038,16039,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,23.24
16039,16040,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,29.99
16040,16041,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,22.83


In [4]:
def get_db_connection():
    conn = sqlite3.connect('my_data.db')
    conn.row_factory = sqlite3.Row
    return conn

In [5]:
con = get_db_connection()

In [23]:
sql = "select * from aaic where id in (%s)" % (', '.join(str(id) for id in [1,2,3]))


In [28]:
post = con.execute(sql).fetchall()
post[0]['id']

1

In [34]:
post = con.execute('SELECT * FROM aaic ORDER BY formatted_price DESC LIMIT 20').fetchall()
post[0]['asin']

'B01IO1GL4U'

In [8]:
p = con.execute('SELECT * from aaic where id = 13055').fetchall()
p[]['brand']

IndexError: list index out of range

In [55]:
pd.read_sql_query('''SELECT * from aaic ORDER BY formatted_price DESC LIMIT 20''',conn)

,id,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,13055,B01IO1GL4U,Brunello Cucinelli,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,brunello cucinelli womens gray satin silk blaz...,995.0
1,1366,B01LBQC1IU,Maison Martin Margiela,Multi-color,https://images-na.ssl-images-amazon.com/images...,SHIRT,maison martin margiela mm6 womens sleeveless b...,99.99
2,1643,B01L7OJBGG,Maison Martin Margiela,Burgundy,https://images-na.ssl-images-amazon.com/images...,SHIRT,maison martin margiela 1 womens burgundy cap s...,99.99
3,2051,B01FUYK5QO,Dolce & Gabbana,Black/ivory,https://images-na.ssl-images-amazon.com/images...,SHIRT,dolce gabbana black ivory striped cotton shor...,99.99
4,2387,B06XSHXWK5,HUGO BOSS,Multi-color,https://images-na.ssl-images-amazon.com/images...,SHIRT,boss esabia windowpane print twist detail jers...,99.99
5,2458,B071JQJFWR,KingYuan,Flesh,https://images-na.ssl-images-amazon.com/images...,SHIRT,kingyuan 105d 42e 46d 48c wider tear drop ov...,99.99
6,2587,B01N8Z2K60,James Perse,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,james perse black shirt 1,99.99
7,2800,B01FUTECN6,Dolce & Gabbana,Blue/White,https://images-na.ssl-images-amazon.com/images...,SHIRT,dolce gabbana blue white striped cotton cap s...,99.99
8,3811,B01EUK0EDS,Ralph Lauren Black Label,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,ralph lauren black label collection denim blac...,99.99
9,3841,B00TA4OJVI,Wild Thing Wild Woman,Black Rainbow,https://images-na.ssl-images-amazon.com/images...,APPAREL,womens black rainbow cascading patch kimono ja...,99.99


In [61]:
pd.read_sql_query('''SELECT * from aaic where product_type_name="SHIRT" GROUP BY ''',conn)

,id,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,1,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies long sleeve stain resistant...,26.26
1,2,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,womens unique 100 cotton special olympics wor...,9.99
2,3,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,featherlite ladies moisture free mesh sport sh...,20.54
3,4,B014ICEJ1Q,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,supernatural chibis sam dean castiel neck tshi...,7.39
4,5,B01NACPBG2,Fifth Degree,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,fifth degree womens gold foil graphic tees jun...,6.95
...,...,...,...,...,...,...,...,...
12822,16038,B01MRV2IFS,YueLian,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,yuelian womens chiffon short sleeves sun prote...,19.25
12823,16039,B01LY4QWLF,Vintage America,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,vintage america womens large lace collared blo...,23.24
12824,16040,B07167SCNH,Tart Collections,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,tart womens collections ann wrap top xs black,29.99
12825,16041,B07575N2WX,Soprano,Gray,https://images-na.ssl-images-amazon.com/images...,SHIRT,soprano womens small tiefringe slubknit tank t...,22.83
